In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [3]:
df = pd.read_csv('北京商圈数据.csv')

In [4]:
df.head()

,cenggao,floor,function_list,jianzhumianji,jinshen,location,loupandizhi,loupanmingcheng,miankuan,release_date,service_price,shangpu_type,shifoufenge,shihejingying,titile,total_price,url,zhuangxiu,shiyongmianji
0,5.40米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",89㎡,9.30米,"东城,王府井,王府井时尚广场",王府井大街(邮编：100006),王府井时尚广场,9.00米,2019/8/1 13:05:10,50.00元/㎡·月,住宅底商,是,"百货超市,酒店宾馆,家居建材,服饰鞋包,生活服务,美容美发,餐饮美食,休闲娱乐",王府井捡漏 业主急用钱价格可谈 纯一层临街 全业态 位置好,963万,https://shop.fang.com/shou/3_436682204.html,精装修,NaN
1,5.40米,第1层(共3层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",105.42㎡,13.00米,"大兴,黄村南,清城",清城北区,清城,8.00米,2019/8/19 18:45:20,12.00元/㎡·月,住宅底商,是,"生活服务,美容美发,餐饮美食",大兴清城 4号线地铁口 8米面宽 户型方正 大产权 随时看房,368万,https://shop.fang.com/shou/3_437610502.html,精装修,NaN
2,4.50米,NaN,"网络,暖气,上水,下水,可明火,排烟,排污,燃气",85.09㎡,10.00米,"大兴,黄村北,枣园小区",枣园小区,枣园小区,8.00米,2019/6/11 10:43:16,10.00元/㎡·月,住宅底商,是,"百货超市,美容美发,餐饮美食,休闲娱乐",枣园巷 纯一层底商 租户黄焖鸡米饭 年租25万 可餐饮,691万,https://shop.fang.com/shou/3_433811445.html,简装修,80.61㎡
3,6.00米,第1层(共1层),"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",135㎡,9.50米,"顺义,后沙峪,宏远航城广场",宏远航城广场,宏远航城广场,13.00米,2019/7/18 12:48:54,6.00元/㎡·月,临街门面,是,"百货超市,酒店宾馆,家居建材,服饰鞋包,生活服务,美容美发,餐饮美食,休闲娱乐",顺义 首都机场核心区 纯一层写字楼底商 业主诚心出售 价格可谈,490万,https://shop.fang.com/shou/3_435954145.html,精装修,NaN
4,3.90米,第3层(共4层),"暖气,停车位,客梯,货梯,扶梯,中央空调,下水,可明火,排烟,排污,外摆区",36㎡,5.00米,"北京周边,涿州,辛集国际皮革城",河北省辛集市辛集国际皮革城,辛集国际皮革城,6.10米,2019/8/26 16:32:20,5.00元/㎡·月,购物中心/百货,是,"餐饮美食, 服饰鞋包",辛集国际皮革城，国际贸易中心,82万,https://shop.fang.com/shou/3_437931411.html,精装修,NaN


In [5]:
lst_lpname = df['loupanmingcheng'].value_counts().index.tolist()

lst_lploc = df['loupandizhi'].value_counts().index.tolist()

In [7]:
city = '北京'
dict_location = {}
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in range(len(lst_lpname)):
    location = lst_lpname[i]
    url = url_pre.format(My_key, location, city)
    res = requests.get(url).json()
    if res['status'] == '0':
        location = lst_lploc[i]
        url = url_pre.format(My_key, location, city)
        res = requests.get(url).json()
        if res['status'] == '0':
            dict_location[lst_lpname[i]] = np.nan
            counter +=1
        else:
            try: 
                dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                if counter % 250 == 0:
                    print('one batch(250) processed')
            except:
                dict_location[lst_lpname[i]] = np.nan
            
    else:
        try: 
            dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            location = lst_lploc[i]
            url = url_pre.format(My_key, location, city)
            res = requests.get(url).json()
            if res['status'] == '0':
                dict_location[lst_lpname[i]] = np.nan
                counter +=1
            else:
                try: 
                    dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                    if counter % 250 == 0:
                        print('one batch(250) processed')
                except:
                    dict_location[lst_lpname[i]] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed
one batch(250) processed


In [8]:
dict_location

{'辛集国际皮革城': '116.909528,39.775560',
 '华远西红世': '116.357763,39.771272',
 '龙湖时代天街': '116.323746,39.686383',
 '五和万科长阳天地': '116.290114,39.815548',
 '万科天地': '116.315631,39.712403',
 '通州万达广场': '116.226850,39.905888',
 '加州水郡': '116.172055,39.766895',
 '朝阳大悦城': '116.523365,39.923063',
 '中粮祥云小镇': '116.536946,40.089356',
 '嘉悦广场': '116.323644,39.786874',
 '富力惠兰星光大道': '116.457757,39.896699',
 '林肯公园': '116.494190,39.804521',
 '金科天宸荟': '116.641364,39.898870',
 '住总正华时代广场': '116.336149,40.080366',
 '欢乐大都汇': '116.108152,39.895530',
 '建外SOHO': '116.458705,39.906458',
 '华清嘉园': '116.335516,39.992202',
 '恒大国际广场': '116.153570,39.717877',
 '华业东方玫瑰': '116.485625,39.918701',
 '星光视界中心': '116.357814,39.772732',
 '富力城星光大道': '116.457757,39.896689',
 '翡翠长安': '116.135382,39.903526',
 '大兴绿地中央广场商业': '116.341395,39.726929',
 '泷悦长安': '116.146438,39.899567',
 '西单大悦城': '116.372980,39.910891',
 '城市之光': '116.589953,39.816294',
 '三里屯Village': '116.454269,39.934685',
 '石景山万达广场': '116.225949,39.905965',
 '五道口国际食尚苑': '116.338808

In [9]:
df['location_coords'] = np.nan

counter = 0
for i in lst_lpname:
    counter += 1
    df_temp = df[df['loupanmingcheng'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,19] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [10]:
df['city'] = city

In [13]:
df.to_csv('北京商铺地址数据.csv', encoding = 'utf-8-sig',index = False)